In [9]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import sys
sys.path.append('..')

dryBean = pd.read_csv('DryBeanDataset/Dry_Bean_Dataset.csv')
dryBean.head()
dryBean.columns[:-1].values

array(['Area', 'Perimeter', 'MajorAxisLength', 'MinorAxisLength',
       'AspectRation', 'Eccentricity', 'ConvexArea', 'EquivDiameter',
       'Extent', 'Solidity', 'roundness', 'Compactness', 'ShapeFactor1',
       'ShapeFactor2', 'ShapeFactor3', 'ShapeFactor4'], dtype=object)

In [2]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
dryBean['Class'] = le.fit_transform(dryBean['Class'])
dryBean['Class'].value_counts()

3    3546
6    2636
5    2027
4    1928
2    1630
0    1322
1     522
Name: Class, dtype: int64

In [3]:
from sklearn.preprocessing import MinMaxScaler

X = dryBean.iloc[:, :-1]
y = dryBean['Class']

X = MinMaxScaler().fit_transform(X)

In [4]:
from sklearn.model_selection import train_test_split

train, test, labels_train, labels_test = train_test_split(X, y,
                                                          train_size=0.80, test_size=0.20,
                                                          random_state=2022)
categorical_features = np.argwhere(
    np.array([len(set(X[:, x])) for x in range(X.shape[1])]) <= 10).flatten()
categorical_features

array([], dtype=int64)

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
import keras
from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers import Activation, Dense, Dropout
from keras.utils import to_categorical

model = RandomForestClassifier(random_state=2022)
# model = AdaBoostClassifier(random_state=2022)
# model = GradientBoostingClassifier(random_state=2022)
# labels_train2 = to_categorical(labels_train, 2)
# labels_test2 = to_categorical(labels_test, 2)
# model = Sequential()
# model.add(Dense(32, activation='relu', input_dim=30))
# model.add(Dropout(0.2))
# model.add(Dense(2, activation='softmax'))
# model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
# model.fit(train, labels_train2, batch_size=32, epochs=50, validation_data=(test, labels_test2))
model.fit(train, labels_train)

Using TensorFlow backend.


RandomForestClassifier(random_state=2022)

In [6]:
pred_origin = model.predict_proba(test)
pred_origin = np.argmax(pred_origin, axis=1)
pred_origin

array([5, 5, 5, ..., 0, 3, 5], dtype=int64)

In [7]:
import lime
import lime.lime_tabular

explainer = lime.lime_tabular.LimeTabularExplainer(train, feature_names=dryBean.columns[:-1].values,
                                                   categorical_features=categorical_features, verbose=False,
                                                   mode='classification', random_state=2022)

In [8]:
exp = explainer.explain_instance(test[25], model.predict_proba, num_features=16,
                                 labels=(0, 1, 2, 3, 4, 5, 6), num_samples=50000)
exp.as_pyplot_figure(label=3)

label=0
label=1


ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 0.0